Application to create candlestick images 
(From https://plot.ly/python/candlestick-charts/)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import os
import numpy as np

In [3]:
debug=False

In [4]:
import plotly.io as pio
# pio.orca.config
from IPython.display import SVG, display
pio.orca.config.use_xvfb = True

In [5]:
pio.orca.config.executable='/opt/conda/envs/fastai/lib/orca_app/orca'

In [6]:
costs=0.008   #cost for trading buy+sell in percent
max_drawdown=0.008

In [7]:
# Set Data Frequency
# The parameter period defines the frequency of the data to be retrieved. 
# minutes: m1, m5, m15 and m30,
# hours: H1, H2, H3, H4, H6 and H8,
# one day: D1,
# one week: W1,
# one month: M1.

request_period='D1'
timestep=dt.timedelta(days=1)

# Value pair to create 224x224 images
# days = 56 | candle width = 4
# days = 32 | candle width = 7
# days = 28 | candle width = 8

#Num_values=56 # Number of periods in one candlestick
#time_diff = dt.timedelta(days=Num_values)
# calculate dimension of candlestick images
# 3 Pixel pro Tag, auch wenn keine Werte für diesen Tag vorliegen.
dimension=224
margin=0
print('Pic dimension: '+str(dimension))

Pic dimension: 224


In [8]:
securities = {'AUD_CHF':56,
              'AUD_JPY':32,
              'AUD_USD':56,
              'AUS200':32,
              'CAD_CHF':56,
              'CAD_JPY':32,
              'EUR_AUD':56,
              'EUR_GBP':56,
              'EUR_JPY':32,
              'EUR_NZD':32,
              'EUR_TRY':56,
              'GBP_AUD':56,
              'GBP_CAD':32,
              'GBP_CHF':32,
              'GBP_JPY':56,
              'GBP_NZD':56,
              'GBP_USD':56,
              'NAS100':32,
              'NZD_CAD':56,
              'NZD_CHF':32,
              'NZD_JPY':56,
              'NZD_USD':32,
              'SPX500':56,
              'US30':32,
              'USD_NOK':32,
              'USD_SEK':56,
              'XAU_USD':56,
            }

In [9]:
path = '/storage/candles/new_20200123'
daily_data_path=path+'/prices_daily/NY-tz-corr'
m30_data_path=path+'/prices_30m/merged'

In [10]:
# make sure output folder for images exist
image_data_path=path+'/images08'
if not os.path.isdir(image_data_path):
    os.mkdir(image_data_path)

In [11]:
files = os.listdir(daily_data_path)
for name in files:
    if name.endswith(".csv"): 
        print(daily_data_path+'/'+name)
        cnt=0
        product_name=name.split('_data')[0]
        Num_values=securities[product_name]
        time_diff = dt.timedelta(days=Num_values)
        # make subdirectory for every product
        akt_image_path = image_data_path+'/'+product_name+'_'+str(Num_values)
        if not os.path.isdir(akt_image_path):
            os.mkdir(akt_image_path)
        candle_values = pd.read_csv(daily_data_path+'/'+name, index_col='date', parse_dates=['date'])
        candle_values=candle_values.tz_localize('UTC').tz_convert(tz='US/Eastern')
        candle_30m_values = pd.read_csv(m30_data_path+'/'+product_name+'_data_20200115.csv', index_col='date', parse_dates=['date'] )        
        candle_30m_values=candle_30m_values.tz_localize('UTC').tz_convert(tz='US/Eastern')
        # set time of start to 00:00
        start = candle_values.index[0].ceil('D')-timestep
        stop = start + time_diff
        num_rows=len(candle_values)
        end_of_data=candle_values.index[num_rows-2]
        #i=1
        while(stop<=end_of_data):        
            #if(debug):print('Timeframe borders '+str(i)+': '+str(start)+' | '+str(stop))
            mask = ((candle_values.index > start) & (candle_values.index <= stop))
            akt_data=candle_values.loc[mask]
            #if(debug):print('Timeframe selected '+str(i)+': '+str(akt_data.index[0])+' | '+str(akt_data.index[-1]))
            #do not produce candlestick, when last day has no data (weekend)
            # This is when last selected element in akt_data is more than 1 timestep before stop
            if(akt_data.index[-1]+timestep>=stop):   
                fig = go.Figure(data=[go.Candlestick(x=akt_data.index,
                                        open=akt_data['open'], high=akt_data['high'],
                                        low=akt_data['low'], close=akt_data['close'],
                                        opacity=0.8, increasing_line_color= 'green', decreasing_line_color= 'red')
                                     ])

                fig.update_layout(xaxis_rangeslider_visible=False, 
                                  width=dimension, height=dimension, 
                                  showlegend=False, xaxis_visible=False, yaxis_visible=False,
                                  paper_bgcolor='#000', plot_bgcolor='#FFF',
                                  margin_l=margin,margin_r=margin, margin_t=margin, margin_b=margin)
                #fig.show()
                # Calculate Label
                label='0'
                movement_tag='XXX'
                if(akt_data.index[-1].hour<=19):
                    start_label = akt_data.index[-1].replace(hour=18, minute=45)
                else:
                    start_label = akt_data.index[-1]+ dt.timedelta(hours=8)
                    start_label = start_label.replace(hour=18, minute=45)    
                end_label = start_label + dt.timedelta(days=4)
                                
                #if(debug):print('Nächster Tag: '+str(start_label))
                
                # Get 30 Min data
                next_day_mask=((candle_30m_values.index > start_label) & (candle_30m_values.index < end_label))
                next_day=candle_30m_values.loc[next_day_mask]
                
                if(len(next_day)>1):
                    # find and fill up missing index values
                    next_day=next_day.resample('30min').ffill()

                    # find first day in 'next_day'
                    first_days=next_day.index[0]
                    end_days=first_days + dt.timedelta(days=1)
                    #if(debug):print('Next day found',first_days)
                        
                    # next 19:00 price
                    mask_1900 = ((next_day.index > first_days.replace(hour=18, minute=45)) & 
                                 (next_day.index < first_days.replace(hour=20, minute=15)))
                    next_day_1900=next_day.loc[mask_1900]
                    
                    if(len(next_day_1900)>0):   # day has 19:00, 19:30 or 20:00 price
                        # find next day 16:00 price
                        mask_1600 = ((next_day.index > end_days.replace(hour=14, minute=45)) & 
                                     (next_day.index < end_days.replace(hour=16, minute=15)))
                        next_day_1600=next_day.loc[mask_1600]
                        if(len(next_day_1600)>0):   # day has has 15:00, 15:30 or 16:00 price
                            # check interval 19:00 - 16:00
                            open=next_day.loc[next_day_1900.index[0]]['open']
                            close_1600=next_day.loc[next_day_1600.index[-1]]['close']
                            #if(debug):print('Next day selected ',next_day_1900.index[0])
                            #if(debug):print('Close 16:00 selected',next_day_1600.index[-1])
                            #Calculate movement
                            movement=int(((close_1600-open)/open)*10000)
                            if(movement>=0):
                                movement_tag = 'P'+ '{:03d}'.format(abs(movement))
                            else:
                                movement_tag = 'M'+ '{:03d}'.format(abs(movement))
                            
                            #Check if movement > costs => Signal
                            if(abs(close_1600-open)/open>=costs):
                                #check for max drawdown
                                mask_1900_1600 = ((next_day.index > first_days.replace(hour=19, minute=00)) & 
                                     (next_day.index < end_days.replace(hour=16, minute=00)))
                                next_day_1900_1600=next_day.loc[mask_1900_1600]
                                max_high=next_day_1900_1600['high'].max()
                                min_low=next_day_1900_1600['low'].min()
                                if(close_1600-open>0): # Long position
                                    if(abs(min_low-open)/open<=max_drawdown): # max drawdown less than 1%
                                        label='L'
                                        cnt +=1
                                else: # short position
                                    if(abs(max_high-open)/open<=max_drawdown): # max drawdown less than 1%
                                        label='S'
                                        cnt +=1

                #if(debug):
                #    if(label!='0'):
                #        print('Open: '+str(open)+' Close: '+str(close_1600)+' Price movement: '+str(abs(close_1600-open)/open)+' Label: '+label)
                #    else:
                #        print('label 0')
                # Create file name including label
                    
                #if(debug):
                #    print('Movement', movement, movement_tag)
                image_name=akt_image_path+'/'+product_name+'_'+str(Num_values)+'_'+start_label.strftime("%Y%m%d")+'_'+movement_tag+'_L'+label+'.png'
                fig.write_image(image_name)
                #print(image_name,'\n')
                #i=i+1
            # For debugging create only a few images
            #if((i==10) & debug):
            #    break
            start = start + timestep
            stop = start + time_diff
        print('Signals: ', cnt)    
            

/storage/candles/new_20200123/prices_daily/NY-tz-corr/USD_SEK_data_D1_20200113_NY_NY_c.csv
Signals:  437
/storage/candles/new_20200123/prices_daily/NY-tz-corr/CAD_CHF_data_D1_20200113_NY_NY_c.csv
Signals:  344
/storage/candles/new_20200123/prices_daily/NY-tz-corr/SPX500_data_D1_20200113_NY_NY_c.csv
Signals:  568
/storage/candles/new_20200123/prices_daily/NY-tz-corr/NZD_CHF_data_D1_20200113_NY_NY_c.csv
Signals:  415
/storage/candles/new_20200123/prices_daily/NY-tz-corr/US30_data_D1_20200113_NY_NY_c.csv
Signals:  538
/storage/candles/new_20200123/prices_daily/NY-tz-corr/EUR_JPY_data_D1_20200113_NY_NY_c.csv
Signals:  381
/storage/candles/new_20200123/prices_daily/NY-tz-corr/GBP_NZD_data_D1_20200113_NY_NY_c.csv
Signals:  390
/storage/candles/new_20200123/prices_daily/NY-tz-corr/CAD_JPY_data_D1_20200113_NY_NY_c.csv
Signals:  464
/storage/candles/new_20200123/prices_daily/NY-tz-corr/GBP_JPY_data_D1_20200113_NY_NY_c.csv
Signals:  438
/storage/candles/new_20200123/prices_daily/NY-tz-corr/GBP_C